In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import seaborn as sns
import re
import matplotlib.pyplot as plt
import string
# File system manangement
import os

In [ ]:
train=pd.read_csv("/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv",encoding='latin1')
test=pd.read_csv("/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv",encoding='latin1')

In [ ]:
train.head()

In [ ]:

train.shape

In [ ]:
test.head()

In [ ]:
test.shape

we will use only two feature which we only need in our classification
(originalTweet,Sentiment)

In [ ]:
train=train[['OriginalTweet','Sentiment']]
test=test[['OriginalTweet','Sentiment']]

let's check name of  classes in Sentiment column

In [ ]:
train['Sentiment'].unique()

then we will display the count of each unique value in Sentiment

In [ ]:
 sns.set(style='darkgrid')
sns.countplot(x=train['Sentiment'],data=train)

we can show the percentage of each Sentiment like that

In [ ]:
plt.pie(train['Sentiment'].value_counts(),labels=(train['Sentiment'].unique()), explode = [0.1,0.1,0.1,0.1,.1],autopct ='%1.1f%%',shadow = True, startangle = 180, labeldistance = 1.5, pctdistance =1.2)

plt.axis('equal')
plt.show()

# Data Cleaning


this line of code will clean all the text ,it will delete every thing except words 
and this will make all words in small 

In [ ]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

# Applying the cleaning function to both test and training datasets
train['OriginalTweet'] = train['OriginalTweet'].apply(lambda x: clean_text(x))
test['OriginalTweet'] = test['OriginalTweet'].apply(lambda x: clean_text(x))

In [ ]:
test.head()

In [ ]:
train.head()

In [ ]:
Neutral_Sentiment=train[train['Sentiment']=='Neutral']['OriginalTweet']
Postive_Sentiment=train[train['Sentiment']=='Positive']['OriginalTweet']
Extremely_Negative_Sentiment=train[train['Sentiment']=='Extremely Negative']['OriginalTweet']
Negative_Sentiment=train[train['Sentiment']=='Negative']['OriginalTweet']
Extremely_Positive_Sentiment=train[train['Sentiment']=='Extremely Positive']['OriginalTweet']
 


let's see wordcloud for each genre of sentimental

In [ ]:
from wordcloud import WordCloud,STOPWORDS

def wordcloud_draw(data, color = 'black'):
     
    wordcloud = WordCloud(stopwords=STOPWORDS,
                      background_color=color,
                      width=2500,
                      height=2000
                     ).generate(''.join(data))
    plt.figure(1,figsize=(13, 13))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()
    


In [ ]:
print("Neutral  words")
wordcloud_draw(Neutral_Sentiment,'white')


as we see above the most words which describe Neutral sentiment are 
(stock, shopping ,home supermarket.......)

In [ ]:
print("Negative words")
wordcloud_draw(Negative_Sentiment,'black')

as we see above the most words which describe Negative sentiment are:
(need, stop, take, price, virus ,pay,...........................)

In [ ]:
print("Postive words")
wordcloud_draw(Postive_Sentiment,'black')

In [ ]:
print("Extremely Negative words")
wordcloud_draw(Extremely_Negative_Sentiment,'black')


as we see above the most words which describe Extremely Negative  sentiment are: (risk,corona virus, scam,panic,...........)

In [ ]:
print("Extremely Positive Sentiment words")
wordcloud_draw(Extremely_Positive_Sentiment,'green')

as we see above the most words which describe Extremely positive  sentiment are: (online,business,keep,still,support,...........)

# Tokenizer 
now we will toknize our original tweet feature ,to vectorize and convert text into sequenses so the network can deel with 

In [ ]:
max_fatures = 3000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(train['OriginalTweet'].values)
X = tokenizer.texts_to_sequences(train['OriginalTweet'].values)
X = pad_sequences(X)

In [ ]:
X.shape

# LSTM
let's try LSTM

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

above we transform our OriginalTweet column into numbers which describe every text  to deel with.

let's transform Sentiment column into numbers

In [ ]:
y=pd.get_dummies(train['Sentiment']).values


In [ ]:
batch_size = 32
model.fit(X, y, epochs = 15, batch_size=batch_size, verbose = 2)


let's prepare our test data 

In [ ]:
test.head()

In [ ]:
max_fatures = 3000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(test['OriginalTweet'].values)
X_test = tokenizer.texts_to_sequences(test['OriginalTweet'].values)
X_test = pad_sequences(X_test)

In [ ]:
y_test=pd.get_dummies(test['Sentiment']).values


In [ ]:
score,acc=model.evaluate(X_test,y_test,verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))